# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

## In the section below, I explored the data to figure out how to approach the exercises. My answers follow this section

In [1]:
from xml.etree import ElementTree as ET
document = ET.parse( './data/mondial_database.xml' )

In [2]:
root = document.getroot()

In [3]:
for elm in root:
    print(elm)

<Element 'country' at 0x105368228>
<Element 'country' at 0x105420f98>
<Element 'country' at 0x10545d8b8>
<Element 'country' at 0x105464ae8>
<Element 'country' at 0x105472048>
<Element 'country' at 0x105472ea8>
<Element 'country' at 0x10547b958>
<Element 'country' at 0x105480728>
<Element 'country' at 0x1054c9ae8>
<Element 'country' at 0x1055228b8>
<Element 'country' at 0x105540a98>
<Element 'country' at 0x10555b4f8>
<Element 'country' at 0x1055b5908>
<Element 'country' at 0x1055d9f48>
<Element 'country' at 0x105622ea8>
<Element 'country' at 0x105628a48>
<Element 'country' at 0x10563f098>
<Element 'country' at 0x1056473b8>
<Element 'country' at 0x105677688>
<Element 'country' at 0x105686ea8>
<Element 'country' at 0x10568ce58>
<Element 'country' at 0x10569a048>
<Element 'country' at 0x1056d9bd8>
<Element 'country' at 0x1057284f8>
<Element 'country' at 0x106071688>
<Element 'country' at 0x10608cf98>
<Element 'country' at 0x10609a048>
<Element 'country' at 0x1060c70e8>
<Element 'country' a

<Element 'mountain' at 0x107536f98>
<Element 'mountain' at 0x10753b188>
<Element 'mountain' at 0x10753b368>
<Element 'mountain' at 0x10753b4f8>
<Element 'mountain' at 0x10753b688>
<Element 'mountain' at 0x10753b8b8>
<Element 'mountain' at 0x10753bae8>
<Element 'mountain' at 0x10753bd18>
<Element 'mountain' at 0x10753bf98>
<Element 'mountain' at 0x1075401d8>
<Element 'mountain' at 0x107540408>
<Element 'mountain' at 0x107540638>
<Element 'mountain' at 0x107540868>
<Element 'mountain' at 0x107540a98>
<Element 'mountain' at 0x107540cc8>
<Element 'mountain' at 0x107540ef8>
<Element 'mountain' at 0x1075451d8>
<Element 'mountain' at 0x107545408>
<Element 'mountain' at 0x107545598>
<Element 'mountain' at 0x1075457c8>
<Element 'mountain' at 0x107545958>
<Element 'mountain' at 0x107545b38>
<Element 'mountain' at 0x107545db8>
<Element 'mountain' at 0x10754a098>
<Element 'mountain' at 0x10754a2c8>
<Element 'mountain' at 0x10754a548>
<Element 'mountain' at 0x10754a7c8>
<Element 'mountain' at 0x107

In [4]:
# get all the country elemens in the root and test if I got it right by printing their names
for country in root.getiterator(tag='country'):
    print(country.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra
France
Spain
Austria
Czech Republic
Germany
Hungary
Italy
Liechtenstein
Slovakia
Slovenia
Switzerland
Belarus
Latvia
Lithuania
Poland
Ukraine
Russia
Belgium
Luxembourg
Netherlands
Bosnia and Herzegovina
Croatia
Bulgaria
Romania
Turkey
Denmark
Estonia
Faroe Islands
Finland
Norway
Sweden
Monaco
Gibraltar
Guernsey
Holy See
Ceuta
Melilla
Iceland
Ireland
San Marino
Jersey
Malta
Isle of Man
Moldova
Portugal
Svalbard
United Kingdom
Afghanistan
China
Iran
Pakistan
Tajikistan
Turkmenistan
Uzbekistan
Armenia
Georgia
Azerbaijan
Bahrain
Bangladesh
Myanmar
India
Bhutan
Brunei
Malaysia
Laos
Thailand
Cambodia
Vietnam
Kazakhstan
North Korea
Kyrgyzstan
Hong Kong
Macao
Mongolia
Nepal
Christmas Island
Cocos Islands
Cyprus
Gaza Strip
Israel
Egypt
Indonesia
Timor-Leste
Papua New Guinea
Iraq
Jordan
Kuwait
Saudi Arabia
Syria
Lebanon
West Bank
Japan
South Korea
Maldives
Oman
United Arab Emirates
Yemen
Philippines
Qatar
Singapore
Sri Lanka
Taiwan
Anguil

In [5]:
# Testing with print function to see if I can access the relevant information: infant morality
print('Country name', ':\t', "infant mortality")
for country in root.getiterator(tag='country'):
    country_name = country.find('name').text
    mort = country.find('infant_mortality')
    if mort == None:
        mort_text = 'unknown'
    else:
        mort_text = mort.text
    print(country_name, ':\t', mort_text)

Country name :	 infant mortality
Albania :	 13.19
Greece :	 4.78
Macedonia :	 7.9
Serbia :	 6.16
Montenegro :	 unknown
Kosovo :	 unknown
Andorra :	 3.69
France :	 3.31
Spain :	 3.33
Austria :	 4.16
Czech Republic :	 2.63
Germany :	 3.46
Hungary :	 5.09
Italy :	 3.31
Liechtenstein :	 4.33
Slovakia :	 5.35
Slovenia :	 4.04
Switzerland :	 3.73
Belarus :	 3.64
Latvia :	 7.91
Lithuania :	 6
Poland :	 6.19
Ukraine :	 8.1
Russia :	 7.08
Belgium :	 4.18
Luxembourg :	 4.28
Netherlands :	 3.66
Bosnia and Herzegovina :	 5.84
Croatia :	 5.87
Bulgaria :	 15.08
Romania :	 10.16
Turkey :	 21.43
Denmark :	 4.1
Estonia :	 6.7
Faroe Islands :	 5.71
Finland :	 3.36
Norway :	 2.48
Sweden :	 2.6
Monaco :	 1.81
Gibraltar :	 6.29
Guernsey :	 3.47
Holy See :	 unknown
Ceuta :	 unknown
Melilla :	 unknown
Iceland :	 3.15
Ireland :	 3.74
San Marino :	 4.52
Jersey :	 3.86
Malta :	 3.59
Isle of Man :	 4.17
Moldova :	 12.93
Portugal :	 4.48
Svalbard :	 unknown
United Kingdom :	 4.44
Afghanistan :	 117.23
China :	 14

In [6]:
# Testing with print function to see if I can access the relevant information: city populations
print('City name', ': ', "Population over the years")
for city in root.getiterator(tag='city'):
    city_name = city.find('name').text
    pops = city.findall('population')
    print(city_name)
    for pop in pops:
        print('\t', pop.get('year'), '\t', pop.text)

City name :  Population over the years
Tirana
	 1987 	 192000
	 1990 	 244153
	 2011 	 418495
Shkodër
	 1987 	 62000
	 2011 	 77075
Durrës
	 1987 	 60000
	 2011 	 113249
Vlorë
	 1987 	 56000
	 2011 	 79513
Elbasan
	 1987 	 53000
	 2011 	 78703
Korçë
	 1987 	 52000
	 2011 	 51152
Komotini
Kavala
	 1981 	 56705
	 1991 	 60187
	 2001 	 63774
	 2011 	 58790
Athina
	 1981 	 885737
	 1991 	 816556
	 2001 	 789166
	 2011 	 664046
Peiraias
	 1981 	 196389
	 1991 	 187399
	 2001 	 181933
	 2011 	 163688
Peristeri
	 1991 	 141971
	 2001 	 146743
	 2011 	 139981
Acharnes
	 1991 	 65035
	 2001 	 82555
	 2011 	 106943
Patra
	 1981 	 142163
	 1991 	 190463
	 2001 	 210494
	 2011 	 213984
Kozani
Kerkyra
	 1991 	 101236
	 2001 	 108652
	 2011 	 102071
Ioannina
	 1991 	 92425
	 2001 	 103101
	 2011 	 112486
Thessaloniki
	 1981 	 406413
	 1991 	 424220
	 2001 	 397156
	 2011 	 325182
Iraklio
	 1981 	 102398
	 1991 	 146536
	 2001 	 163115
	 2011 	 173993
Chania
	 1991 	 83712
	 2001 	 98202
	 2011 	 108

	 2000 	 588994
	 2011 	 571143
Duisburg
	 1987 	 525378
	 1992 	 539094
	 2000 	 514915
	 2011 	 488468
Bochum
	 1987 	 386271
	 1992 	 400356
	 2000 	 391147
	 2011 	 362286
Wuppertal
	 1987 	 365662
	 1992 	 388102
	 2000 	 366434
	 2011 	 342661
Bielefeld
	 1987 	 305566
	 1992 	 324287
	 2000 	 321758
	 2011 	 326870
Gelsenkirchen
	 1987 	 287508
	 1992 	 295368
	 2000 	 278695
	 2011 	 258766
Bonn
	 1987 	 276653
	 1992 	 298227
	 2000 	 302247
	 2011 	 305765
Mönchengladbach
	 1987 	 249587
	 1992 	 265069
	 2000 	 263014
	 2011 	 255188
Münster
	 1987 	 246186
	 1992 	 267072
	 2000 	 265609
	 2011 	 289576
Krefeld
	 1987 	 232261
	 2011 	 222247
Aachen
	 1987 	 229740
	 2011 	 236420
Oberhausen
	 1987 	 220286
	 2011 	 210216
Hagen
	 1987 	 209363
	 2011 	 187944
Hamm
	 1987 	 171170
	 2011 	 176037
Herne
	 1987 	 174238
	 2011 	 155160
Mülheim an der Ruhr
	 1987 	 176423
	 2011 	 166865
Solingen
	 1987 	 159103
	 2011 	 155265
Leverkusen
	 1987 	 154692
	 2011 	 158984
Neuss


	 2012 	 362286
Toruń
	 1990 	 201000
	 2002 	 211243
	 2005 	 208278
	 2009 	 205934
	 2012 	 204847
Grudziadz
	 1990 	 101000
	 2002 	 100376
	 2012 	 98726
Włocławek
	 1990 	 121000
	 2002 	 121229
	 2005 	 120369
	 2009 	 117785
	 2012 	 115982
Lublin
	 1990 	 350000
	 2002 	 357110
	 2005 	 355998
	 2009 	 350392
	 2012 	 348120
Zielona Góra
	 1990 	 113000
	 2002 	 118293
	 2005 	 118516
	 2009 	 117557
	 2012 	 119182
Gorzów Wielkopolski
	 1990 	 123000
	 2002 	 125914
	 2005 	 125578
	 2009 	 125149
	 2012 	 124470
Łódź
	 1990 	 852000
	 2002 	 789318
	 2005 	 774004
	 2009 	 744541
	 2012 	 722022
Kraków
	 1990 	 748000
	 2002 	 758544
	 2005 	 757430
	 2009 	 754854
	 2012 	 759131
Tarnów
	 1990 	 120000
	 2002 	 119913
	 2005 	 118267
	 2009 	 115341
	 2012 	 113188
Warszawa
	 1990 	 1655000
	 2002 	 1671670
	 2005 	 1692854
	 2009 	 1711466
	 2012 	 1711324
Płock
	 1990 	 122000
	 2002 	 128361
	 2005 	 127655
	 2009 	 126675
	 2012 	 124048
Radom
	 1990 	 226000
	 2002 	 2

	 2002 	 141000
	 2010 	 207425
Odincovo
	 1979 	 101365
	 1989 	 125149
	 2002 	 134844
	 2010 	 138930
Orechovo-Zujevo
	 1979 	 132301
	 1989 	 137198
	 2002 	 122248
	 2010 	 120670
Noginsk
	 1979 	 118750
	 1989 	 123020
	 2002 	 117555
	 2010 	 100072
Ščëlkovo
	 1979 	 100281
	 1989 	 109225
	 2002 	 112865
	 2010 	 110411
Sergijev Posad
	 1979 	 107144
	 1989 	 114696
	 2002 	 113581
	 2010 	 111179
Korolëv
	 1979 	 133470
	 1989 	 159363
	 2002 	 170558
	 2010 	 183402
Žukovskij
	 1979 	 90288
	 1989 	 100609
	 2002 	 101328
	 2010 	 104736
Puškino
	 1979 	 68518
	 1989 	 75847
	 2002 	 72425
	 2010 	 102874
Orel
	 1979 	 304971
	 1989 	 336862
	 2002 	 333310
	 2010 	 317747
	 2013 	 318136
Ryazan
	 1979 	 453267
	 1989 	 514638
	 2002 	 524104
	 2010 	 524927
	 2013 	 527905
Smolensk
	 1979 	 276402
	 1989 	 341483
	 2002 	 325137
	 2010 	 326861
	 2013 	 330970
Tver
	 1979 	 411548
	 1989 	 450941
	 2002 	 408903
	 2010 	 403606
	 2013 	 408877
Tula
	 1979 	 514008
	 1989 	 5

Novosibirsk
	 1979 	 1312480
	 1989 	 1436516
	 2002 	 1425508
	 2010 	 1473754
	 2013 	 1523801
Omsk
	 1979 	 1014246
	 1989 	 1148418
	 2002 	 1156849
	 2010 	 1154116
	 2013 	 1160670
Tomsk
	 1979 	 420730
	 1989 	 501963
	 2002 	 487838
	 2010 	 524669
	 2013 	 547989
Seversk
	 2002 	 109106
	 2010 	 108590
Tyumen
	 1979 	 358992
	 1989 	 476869
	 2002 	 510719
	 2010 	 581907
	 2013 	 634171
Ulan-Ude
	 1979 	 300370
	 1989 	 352530
	 2002 	 359391
	 2010 	 404426
	 2013 	 416079
Kyzyl
	 1979 	 66027
	 1989 	 84641
	 2002 	 104105
	 2010 	 109918
Abakan
	 1979 	 128311
	 1989 	 154092
	 2002 	 165197
	 2010 	 165214
Krasnoyarsk
	 1979 	 796305
	 1989 	 912629
	 2002 	 909341
	 2010 	 973826
	 2013 	 1016385
Noril'sk
	 1979 	 180358
	 1989 	 174673
	 2002 	 134832
	 2010 	 175365
Ačinsk
	 1979 	 116854
	 1989 	 121572
	 2002 	 118744
	 2010 	 109155
Irkutsk
	 1979 	 549787
	 1989 	 626135
	 2002 	 593604
	 2010 	 587891
	 2013 	 606137
Angarsk
	 1979 	 238802
	 1989 	 265835
	 2002 

...

	 2014 	 2344734
Buenaventura
	 1995 	 266988
	 2005 	 324207
	 2014 	 392054
Palmira
	 1995 	 256823
	 2005 	 278358
	 2014 	 302727
Tuluá
	 1995 	 138124
	 2005 	 183236
	 2014 	 209086
Cartago
	 1995 	 117166
	 2005 	 121741
	 2014 	 131545
Mitu
Puerto Carreno
San Juan
	 2000 	 421958
	 2010 	 381931
Ponce
	 2000 	 155038
	 2010 	 132502
Bayamón
	 2000 	 203499
	 2010 	 185996
Carolina
	 2000 	 168164
	 2010 	 157832
Basseterre
	 1987 	 14700
	 2005 	 15500
Castries
	 2010 	 3661
Saint-Pierre
	 2009 	 5618
Kingstown
	 2012 	 12909
Port-of-Spain
	 2000 	 548299
Grand Turk
Charlotte Amalie
	 1990 	 12331
	 2000 	 11004
	 2010 	 10354
Pago Pago
Newcastle
	 1990 	 429000
	 1996 	 463400
	 2001 	 492549
	 2011 	 540002
Sydney
	 1990 	 3657000
	 1996 	 3881100
	 2001 	 4128272
	 2011 	 4605992
Wollongong
	 1990 	 238000
	 1996 	 255700
	 2001 	 269597
	 2011 	 288101
Sunshine Coast
	 1990 	 110000
	 1996 	 156400
	 2001 	 186391
	 2011 	 241643
Darwin
	 1987 	 68500
	 1996 	 95800
	 2001 

	 1991 	 227759
	 1996 	 271324
	 2000 	 289183
	 2010 	 344427
Uberaba
	 1991 	 200705
	 1996 	 232134
	 2000 	 244171
	 2010 	 289376
Governador Valadares
	 1991 	 215098
	 1996 	 231077
	 2000 	 236098
	 2010 	 253300
Ipatinga
	 1991 	 178830
	 1996 	 195736
	 2000 	 210895
	 2010 	 236968
Divinópolis
	 1991 	 144429
	 1996 	 171562
	 2000 	 177973
	 2010 	 207516
Sete Lagoas
	 1991 	 140125
	 1996 	 167112
	 2000 	 180785
	 2010 	 208956
Teófilo Otoni
	 1991 	 101966
	 1996 	 127214
	 2000 	 102812
	 2010 	 110076
Poços de Caldas
	 1991 	 105205
	 1996 	 121845
	 2000 	 130826
	 2010 	 148722
Patos de Minas
	 1991 	 87403
	 1996 	 112690
	 2000 	 111333
	 2010 	 127724
Barbacena
	 1991 	 83319
	 1996 	 107843
	 2000 	 103669
	 2010 	 115568
Sabará
	 1991 	 74757
	 1996 	 100570
	 2000 	 112694
	 2010 	 123084
Varginha
	 1991 	 82242
	 1996 	 100079
	 2000 	 104165
	 2010 	 119061
Santa Luzia
	 1991 	 130186
	 1996 	 152682
	 2000 	 184208
	 2010 	 202378
Belém
	 1991 	 849187
	 199

	 2010 	 313046
Limeira
	 1991 	 177934
	 1996 	 230292
	 2000 	 238349
	 2010 	 267785
Itaquaquecetuba
	 1991 	 164957
	 1996 	 228533
	 2000 	 272942
	 2010 	 321770
Guarujá
	 1991 	 210192
	 1996 	 226185
	 2000 	 264733
	 2010 	 290696
Taubaté
	 1991 	 197801
	 1996 	 220179
	 2000 	 229855
	 2010 	 272673
Embu
	 1991 	 155990
	 1996 	 195676
	 2000 	 207663
	 2010 	 240230
Taboão da Serra
	 1991 	 160084
	 1996 	 182441
	 2000 	 197644
	 2010 	 244528
Suzano
	 1991 	 152003
	 1996 	 180703
	 2000 	 221423
	 2010 	 253240
Marília
	 1991 	 150520
	 1996 	 177503
	 2000 	 189719
	 2010 	 207021
Presidente Prudente
	 1991 	 160227
	 1996 	 177236
	 2000 	 185229
	 2010 	 203375
Barueri
	 1991 	 130799
	 1996 	 176740
	 2000 	 208281
	 2010 	 240749
São Carlos
	 1991 	 148408
	 1996 	 175295
	 2000 	 183433
	 2010 	 213061
Jacareí
	 1991 	 157026
	 1996 	 168030
	 2000 	 183377
	 2010 	 208297
Americana
	 1991 	 153653
	 1996 	 167901
	 2000 	 182159
	 2010 	 209654
Araraquara
	 1991 	

	 2012 	 118240
Diffa
Dosso
Maradi
	 1988 	 110005
	 2001 	 148017
	 2012 	 267249
Tessaoua
	 2001 	 119506
	 2012 	 172796
Aguié
	 2001 	 101957
	 2012 	 152788
Niamey
	 1988 	 391876
	 2001 	 725030
	 2012 	 1026848
Tahoua
	 1988 	 49948
	 2001 	 88761
	 2012 	 149498
Birni-N'Konni
	 2001 	 103249
	 2012 	 149414
Illéla
	 2001 	 91312
	 2012 	 142214
Tillaberi
Zinder
	 1988 	 119827
	 2001 	 214945
	 2012 	 322935
Tanout
	 2001 	 95626
	 2012 	 154238
Magaria
	 2001 	 75876
	 2012 	 130707
Tunis
	 2013 	 651284
Sfax
	 2013 	 299235
Sousse
	 2013 	 223844
El Aaiun
	 1994 	 136950
	 2004 	 183691
Cabinda
Mbanza Congo
Uige
Luanda
	 1960 	 189500
	 1970 	 475328
	 1983 	 898000
	 1987 	 1136000
	 2000 	 2571600
	 2011 	 5000000
Ndalatando
Sumbe
Malanje
Lucapa
Saurimo
Benguela
	 1983 	 155000
	 2006 	 513000
Huambo
Bie
Luena
Menongue
Namibe
Lubango
Caxito
Ngiva
Dolisie
	 2007 	 83798
	 2010 	 128032
Sibiti
Madingou
Kinkala
Djambala
Owando
Ewo
Ouesso
Impfondo
Brazzaville
	 2007 	 1373382
	

	 2003 	 41557
Conakry
	 2014 	 1667864
N'Zérékoré
	 2014 	 194178
Monrovia
	 1984 	 421053
	 2008 	 1010970
Djibouti
	 1989 	 450000
	 1991 	 330000
	 2009 	 475322
Asmara
	 1997 	 380568
Addis Ababa
	 1994 	 2084588
	 2007 	 2738248
	 2012 	 3040740
Aysaita
Bahir Dar
	 1994 	 96140
	 2007 	 220344
	 2012 	 191016
Gondar
	 1994 	 112249
	 2007 	 206987
	 2012 	 254450
Dessie
	 1994 	 97314
	 2007 	 151094
	 2012 	 147592
Asosa
Dire Dawa
	 1994 	 164851
	 2007 	 342827
	 2012 	 262884
Gambella
Harar
	 1994 	 76378
	 2007 	 183344
	 2012 	 110457
Finfine
Adama
	 1994 	 127842
	 2007 	 222035
	 2012 	 271562
Jimma
	 1994 	 88867
	 2007 	 120600
	 2012 	 149166
Bishoftu
	 1994 	 73372
	 2007 	 100114
	 2012 	 123230
Jijiga
	 2012 	 147482
Mekele
	 1994 	 96938
	 2007 	 215546
	 2012 	 273601
Hawassa
	 1994 	 69169
	 2007 	 259803
	 2012 	 212665
Mogadishu
Mombasa
	 1989 	 465000
	 1991 	 600000
	 1999 	 665018
	 2009 	 938131
Malindi
	 1989 	 35200
	 1999 	 53805
	 2009 	 118265
Machakos


In [7]:
# Testing with print function to see if I can access the relevant information: country populations
print('Country name', ': ', "Population over the years")
for country in root.getiterator(tag='country'):
    country_name = country.find('name').text
    pops = country.findall('population')
    print(country_name)
    for pop in pops:
        print('\t', pop.get('year'), '\t', pop.text)

Country name :  Population over the years
Albania
	 1950 	 1214489
	 1960 	 1618829
	 1970 	 2138966
	 1980 	 2734776
	 1990 	 3446882
	 1997 	 3249136
	 2000 	 3304948
	 2001 	 3069275
	 2011 	 2800138
Greece
	 1861 	 1096810
	 1870 	 1457894
	 1879 	 1679470
	 1896 	 2433806
	 1907 	 2631592
	 1920 	 5016889
	 1928 	 6204684
	 1940 	 7344860
	 1951 	 7632801
	 1961 	 8388553
	 1971 	 8768372
	 1981 	 9739589
	 1991 	 10217335
	 2001 	 10934097
	 2011 	 10816286
Macedonia
	 1921 	 808724
	 1931 	 949958
	 1948 	 1152986
	 1953 	 1304514
	 1961 	 1406003
	 1971 	 1647308
	 1981 	 1909136
	 1991 	 2033964
	 1994 	 1935034
	 2002 	 2022547
	 2011 	 2059794
Serbia
	 1950 	 6732256
	 1960 	 7657958
	 1970 	 8236185
	 1980 	 9057483
	 1990 	 9735429
	 1991 	 7620531
	 2002 	 7498001
	 2011 	 7120666
Montenegro
	 1921 	 311341
	 1931 	 360044
	 1948 	 377189
	 1953 	 419873
	 1961 	 471894
	 1971 	 529604
	 1981 	 584310
	 1991 	 615035
	 2003 	 620145
	 2011 	 620029
Kosovo
	 1981 	 1584440

	 1980 	 3250924
	 1985 	 3570000
	 1995 	 4581258
	 2005 	 5621982
	 2012 	 6514432
Thailand
	 1909 	 8149487
	 1919 	 9207355
	 1929 	 11506207
	 1937 	 14464105
	 1947 	 17442689
	 1956 	 20095130
	 1960 	 26257916
	 1970 	 34397374
	 1980 	 44824540
	 1990 	 54548530
	 2000 	 60916441
	 2010 	 65981659
Cambodia
	 1950 	 4432716
	 1960 	 5720019
	 1962 	 5728771
	 1998 	 11437656
	 2008 	 13395682
	 2010 	 14364931
Vietnam
	 1950 	 24949467
	 1960 	 32912298
	 1970 	 43782542
	 1980 	 54897387
	 1990 	 68909883
	 1999 	 76327900
	 2009 	 85846997
	 2012 	 88772900
Kazakhstan
	 1950 	 6702996
	 1959 	 9303801
	 1970 	 13026274
	 1979 	 14709508
	 1989 	 16232324
	 1999 	 14981281
	 2009 	 16009597
	 2014 	 17165239
North Korea
	 1925 	 6014099
	 1930 	 6537303
	 1940 	 7920279
	 1944 	 8554857
	 1993 	 20522351
	 2008 	 23349859
	 2010 	 24500520
Kyrgyzstan
	 1950 	 1740000
	 1959 	 2065837
	 1979 	 3522832
	 1989 	 4257755
	 1999 	 4850734
	 2009 	 5107640
	 2014 	 5776600
Hong Kong

	 1997 	 3463083
	 2000 	 3810179
	 2011 	 4301712
	 2014 	 4773119
Nicaragua
	 1950 	 1057023
	 1963 	 1535588
	 1971 	 1877952
	 1995 	 4357099
	 2005 	 5142098
	 2012 	 6071045
Panama
	 1911 	 336742
	 1921 	 446098
	 1930 	 467459
	 1940 	 622576
	 1950 	 805285
	 1960 	 1075541
	 1970 	 1428082
	 1980 	 1805287
	 1990 	 2329329
	 1997 	 2655094
	 2000 	 2839177
	 2010 	 3405813
Cuba
	 1950 	 5919997
	 1960 	 7141130
	 1970 	 8715054
	 1980 	 9835085
	 1990 	 10600841
	 1997 	 10951334
	 2000 	 11138416
	 2002 	 11177743
	 2012 	 11167325
Dominica
	 1871 	 27178
	 1881 	 28211
	 1891 	 26841
	 1901 	 28894
	 1911 	 33863
	 1921 	 37059
	 1946 	 47624
	 1960 	 59916
	 1970 	 69549
	 1981 	 73795
	 1991 	 71183
	 2001 	 71242
	 2011 	 70739
Dominican Republic
	 1950 	 2380116
	 1960 	 3311788
	 1970 	 4523917
	 1980 	 5825715
	 1990 	 7245127
	 1993 	 7293390
	 1997 	 8088881
	 2002 	 8562541
	 2010 	 9445281
Haiti
	 1950 	 3097220
	 1971 	 4329991
	 1982 	 5053189
	 1995 	 7180294
	

	 1985 	 2955241
	 1996 	 3163763
	 2004 	 3241003
	 2011 	 3286314
Peru
	 1950 	 7632461
	 1960 	 9931518
	 1961 	 10420357
	 1972 	 14121564
	 1981 	 17762231
	 1993 	 22048356
	 1997 	 24523408
	 2007 	 27412157
	 2012 	 30135875
French Guiana
	 1993 	 157213
	 2006 	 208171
	 2011 	 237549
Guyana
	 1946 	 375701
	 1960 	 560330
	 1970 	 701718
	 1980 	 759567
	 1991 	 723673
	 2002 	 751223
	 2012 	 747884
Suriname
	 1950 	 183681
	 1964 	 324211
	 1972 	 379607
	 1980 	 355240
	 1997 	 436418
	 2003 	 481146
	 2004 	 492829
	 2012 	 541638
Venezuela
	 1950 	 5093708
	 1960 	 7580281
	 1970 	 10724328
	 1980 	 15096432
	 1990 	 18105265
	 2001 	 23054210
	 2011 	 28946101
Ecuador
	 1950 	 3452498
	 1960 	 4514593
	 1970 	 6024764
	 1980 	 7909432
	 1990 	 9697979
	 1997 	 11466291
	 2001 	 12156608
	 2010 	 14483499
Falkland Islands
	 1901 	 2043
	 1911 	 2272
	 1921 	 2094
	 1931 	 2392
	 1946 	 2239
	 1953 	 2230
	 1962 	 2172
	 1972 	 1957
	 1986 	 1878
	 1991 	 2091
	 1996 	 25

In [8]:
# Testing with print function to see if I can access the relevant information: ethnic groups
print('Country name', ': ', "Ethnic Groups")
for country in root.getiterator(tag='country'):
    country_name = country.find('name').text
    egroup = country.findall('ethnicgroup')
    print(country_name)
    for e in egroup:
        print('\t', e.get('percentage'), '\t', e.text)

Country name :  Ethnic Groups
Albania
	 95 	 Albanian
	 3 	 Greek
Greece
	 93 	 Greek
Macedonia
	 64.2 	 Macedonian
	 25.2 	 Albanian
	 3.9 	 Turkish
	 2.7 	 Gypsy
	 1.8 	 Serb
Serbia
	 82.9 	 Serb
	 0.9 	 Montenegrin
	 3.9 	 Hungarian
	 1.4 	 Roma
	 1.8 	 Bosniak
	 1.1 	 Croat
Montenegro
	 43 	 Montenegrin
	 32 	 Serb
	 8 	 Bosniak
	 5 	 Albanian
Kosovo
	 92 	 Albanian
	 5 	 Serbian
Andorra
	 43 	 Spanish
	 33 	 Andorran
	 11 	 Portuguese
	 2 	 French
	 5 	 African
France
Spain
	 100 	 Mediterranean Nordic
Austria
	 91.1 	 Austrian
	 1.6 	 Turkish
	 1 	 Slovene
	 2 	 Croat
	 2 	 Serbs
	 0.9 	 German
Czech Republic
	 90.4 	 Czech
	 3.7 	 Moravian
	 1.9 	 Slovak
	 0.5 	 German
	 0.6 	 Polish
	 0.3 	 Roma
	 0.2 	 Hungarian
Germany
	 91.5 	 German
	 2.4 	 Turkish
Hungary
	 92.3 	 Hungarian
	 1.9 	 Roma
	 2 	 Serb
	 2.6 	 German
	 0.8 	 Slovak
	 0.7 	 Romanian
Italy
Liechtenstein
	 5 	 Italian
	 95 	 Alemannic
Slovakia
	 85.7 	 Slovak
	 9.7 	 Hungarian
	 1.7 	 Roma
	 1 	 Czech
	 0.3 	 Ukra

In [9]:
# Testing with print function to see if I can access the relevant information: lengths of rivers
print('Country\tRiver name\tLength')
for river in root.getiterator(tag='river'):
    river_name = river.find('name').text
    length = river.find('length').text
    country = river.get('country')
    print(country, '\t', river_name, '\t', length)


Country	River name	Length
IS 	 Thjorsa 	 230
IS 	 Joekulsa a Fjoellum 	 206
N 	 Glomma 	 604
N 	 Lagen 	 322
S 	 Goetaaelv 	 93
N S 	 Klaraelv 	 460
S 	 Umeaelv 	 470
S 	 Dalaelv 	 520
S 	 Vaesterdalaelv 	 320
S 	 Oesterdalaelv 	 241
SF N R 	 Paatsjoki 	 145
SF 	 Ounasjoki 	 300
SF 	 Kemijoki 	 550
SF 	 Oulujoki 	 107
SF 	 Kymijoki 	 203
SF 	 Kokemaeenjoki 	 121
SF R 	 Vuoksi 	 162
GB 	 Thames 	 346
NL B F 	 Maas 	 925
F 	 Loire 	 1013
F E 	 Garonne 	 647
F CH 	 Rhone 	 812
F 	 Saone 	 480
F CH 	 Doubs 	 453
F 	 Isere 	 290
F 	 Seine 	 776
F 	 Marne 	 514
E P 	 Tajo 	 1007
E P 	 Douro 	 897
E P 	 Guadiana 	 742
E 	 Guadalquivir 	 657
E 	 Ebro 	 925
I 	 Po 	 652
CH I 	 Ticino 	 248
I 	 Adda 	 313
I 	 Mincio 	 75
I 	 Etsch 	 415
I 	 Tiber 	 405
I 	 Arno 	 240
SRB A D H HR SK BG RO MD UA 	 Donau 	 2845
D 	 Breg 	 45.9
D 	 Brigach 	 43
D A 	 Iller 	 147
D A 	 Lech 	 264
D A 	 Isar 	 295
D 	 Ammer 	 168
D 	 Würm 	 35
CH A D 	 Inn 	 517
D 	 Alz 	 150
A D 	 Salzach 	 225
A 	 Enns 	 254
CZ SK 

AttributeError: 'NoneType' object has no attribute 'text'

In [10]:
# Testing with print function to see if I can access the relevant information: sizes of lakes
print('Country\tLake name\tArea')
for lake in root.getiterator(tag='lake'):
    lake_name = lake.find('name').text
    area = river.find('area').text
    country = lake.get('country')
    print(country, '\t', lake_name, '\t', area)

Country	Lake name	Area
SF 	 Inari 	 3254853
SF 	 Oulujaervi 	 3254853
SF 	 Kallavesi 	 3254853
SF 	 Saimaa 	 3254853
SF 	 Paeijaenne 	 3254853
N 	 Mjoesa-See 	 3254853
S 	 Storuman 	 3254853
S 	 Siljan 	 3254853
S 	 Maelaren 	 3254853
S 	 Vaenern 	 3254853
S 	 Vaettern 	 3254853
DK 	 Arresoe 	 3254853
GB 	 Loch Ness 	 3254853
GB 	 Loch Lomond 	 3254853
A D CH 	 Bodensee 	 3254853
D 	 Chiemsee 	 3254853
D 	 Starnberger See 	 3254853
D 	 Ammersee 	 3254853
D 	 Laacher Maar 	 3254853
F CH 	 Lac Leman 	 3254853
CH 	 Zurichsee 	 3254853
CH 	 Thunersee 	 3254853
CH 	 Brienzersee 	 3254853
CH 	 Vierwaldstattersee 	 3254853
I CH 	 Lago Maggiore 	 3254853
I 	 Lago di Como 	 3254853
I 	 Lago di Garda  	 3254853
I 	 Lago Trasimeno 	 3254853
I 	 Lago di Bolsena 	 3254853
I 	 Lago di Bracciano 	 3254853
E 	 Laguna de Gallocanta 	 3254853
A H 	 Neusiedlersee 	 3254853
H 	 Balaton 	 3254853
AL MNE 	 Lake Skutari 	 3254853
AL MK GR 	 Lake Prespa 	 3254853
AL MK 	 Lake Ohrid 	 3254853
UA 	 Kiev Reservo

In [11]:
# Testing with print function to see if I can access the relevant information: elevation of airports
print('Country\tAirport\tElevation')
for ap in root.getiterator(tag='airport'):
    ap_name = ap.find('name').text
    ele = ap.find('elevation').text
    country = ap.get('country')
    print(country, '\t', ap_name, '\t', ele)

Country	Airport	Elevation
AFG 	 Herat 	 977
AFG 	 Kabul Intl 	 1792
AL 	 Tirana Rinas 	 38
DZ 	 Cheikh Larbi Tebessi 	 811
DZ 	 Batna Airport 	 822
DZ 	 Soummam 	 6
DZ 	 Tamanrasset 	 1377
DZ 	 Biskra 	 88
DZ 	 Mohamed Boudiaf Intl 	 691
DZ 	 Ain Arnat Airport 	 1024
DZ 	 Es Senia 	 90
DZ 	 Noumerat 	 461
DZ 	 Annaba 	 5
DZ 	 Houari Boumediene 	 25
DZ 	 Zenata 	 248
AMSA 	 Pago Pago Intl 	 10
ANG 	 Lubango 	 1762
ANG 	 Cabinda 	 20
ANG 	 Menongue 	 1363
ANG 	 Luanda 4 De Fevereiro 	 74
ANG 	 Huambo 	 1703
AXA 	 Wallblake 	 39
AG 	 V C Bird Intl 	 19
RA 	 La Rioja 	 438
RA 	 Jujuy 	 920
RA 	 Comandante Espora 	 75
RA 	 Teniente Benjamin Matienzo 	 456
RA 	 San Luis 	 710
RA 	 Santiago del Estero 	 200
RA 	 Sauce Viejo 	 17
RA 	 Corrientes 	 62
RA 	 Presidente Peron 	 273
RA 	 Salta 	 1246
RA 	 Aeroparque Jorge Newbery 	 5
RA 	 Ministro Pistarini 	 20
RA 	 Ushuaia Malvinas Argentinas 	 22
RA 	 Formosa 	 59
RA 	 Posadas 	 131
RA 	 Rosario 	 26
RA 	 Resistencia 	 53
RA 	 Rio Gallegos 	 19


J 	 Yamagata 	 108
J 	 Fukuoka 	 10
J 	 Fukushima Airport 	 372
J 	 Hiroshima 	 332
J 	 Tokyo Intl 	 11
J 	 Hakodate 	 46
J 	 Saga Airport 	 2
J 	 Osaka Intl 	 15
J 	 Akita 	 95
J 	 Toyama 	 29
J 	 Aomori 	 202
J 	 New Chitose 	 25
J 	 Takamatsu 	 185
J 	 Sendai 	 5
J 	 Kansai 	 15
J 	 Niigata 	 2
J 	 Kagoshima 	 276
J 	 Tottori 	 20
J 	 Miyazaki 	 6
J 	 Komatsu 	 11
J 	 Kumamoto 	 196
J 	 Tokushima 	 8
J 	 Kochi 	 13
J 	 New Kitakyushu 	 6
J 	 Chubu Centrair Intl 	 5
J 	 Nagasaki 	 5
J 	 Kobe 	 7
J 	 Narita Intl 	 43
J 	 Yamaguchi Ube 	 7
J 	 Matsuyama 	 8
J 	 Mt. Fuji Shizuoka Airport 	 132
GBJ 	 Jersey 	 84
JOR 	 Queen Alia Intl 	 730
KAZ 	 Semipalatinsk 	 232
KAZ 	 Uralsk 	 38
KAZ 	 Atyrau 	 -22
KAZ 	 Aktau 	 6
KAZ 	 Zhezkazgan Airport 	 381
KAZ 	 Kostanay West Airport 	 181
KAZ 	 Ust Kamenogorsk Airport 	 286
KAZ 	 Shymkent 	 422
KAZ 	 Aktyubinsk 	 225
KAZ 	 Almaty 	 681
KAZ 	 Astana Intl 	 355
KAZ 	 Sary-Arka 	 538
KAZ 	 Pavlodar 	 125
KAZ 	 Kzylorda 	 131
EAK 	 Malindi Airport 	

E 	 Malaga 	 16
E 	 Bilbao 	 42
E 	 Tenerife Sur 	 64
E 	 Talavera La Real 	 186
E 	 Alicante 	 43
E 	 Barcelona 	 4
E 	 Lanzarote 	 14
E 	 Vigo 	 261
MEL 	 Melilla 	 48
E 	 Murcia San Javier 	 3
E 	 La Gomera Airport 	 219
E 	 Fuerteventura 	 25
E 	 Jerez 	 28
E 	 Son Sant Joan 	 7
E 	 Pamplona 	 459
E 	 Granada 	 567
E 	 Ibiza 	 6
E 	 Zaragoza Ab 	 263
E 	 Reus 	 71
E 	 San Sebastian 	 5
E 	 Barajas 	 610
E 	 Menorca 	 91
E 	 Valencia 	 69
E 	 Valladolid 	 846
E 	 Sevilla 	 34
E 	 A Coruna 	 99
E 	 Almeria 	 21
E 	 Santiago 	 370
CL 	 Bandaranaike Intl Colombo 	 9
SUD 	 Khartoum 	 386
SME 	 Johan A Pengel Intl 	 18
SD 	 Matsapha 	 633
S 	 Kronoberg 	 186
S 	 Visby 	 50
S 	 Kungsangen 	 10
S 	 Kallax 	 20
S 	 Saab 	 52
S 	 Angelholm-Helsingborg Airport 	 21
S 	 Kristianstad 	 23
S 	 Bromma 	 14
S 	 Kalmar 	 5
S 	 Umea 	 7
S 	 Skavsta 	 43
S 	 Vasteras 	 6
S 	 Halmstad 	 31
S 	 Jonkoping 	 226
S 	 Östersund Airport 	 376
S 	 Örebro 	 57
S 	 Landvetter 	 154
S 	 Save 	 18
S 	 Arlanda 	 

****
# My Anwers to the Exercises

In [12]:
# Importing Modules and reading in the data
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

In [13]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

****
# Exercise1: 10 countries with the lowest infant mortality rates

In [14]:
countries = []      # list to hold country names
mortalities = []    # list to hold infant mortalities

# Find elements tagged with 'country', iterate over all such elements
for country in root.getiterator(tag='country'):
    
    # Add country name to the list
    countries.append(country.find('name').text)
    
    # Add infant mortality to the list
    mort = country.find('infant_mortality')
    if mort == None:           #if data don't exist, insert None
        mortality = None
    else:
        mortality = float(mort.text)     #if data exist, convert str to float
    mortalities.append(mortality)

In [27]:
# Create DataFrame with the above list, list lowest top10 countries
d = {'country': countries, 'infant_mortality': mortalities}
imort = pd.DataFrame(d)
d1 = imort.sort_values('infant_mortality', ascending=True).head(10)
d1['Rank'] = [i for i in range(1, 11)]
d1

,country,infant_mortality,Rank
38,Monaco,1.81,1
98,Japan,2.13,2
117,Bermuda,2.48,3
36,Norway,2.48,4
106,Singapore,2.53,5
37,Sweden,2.60,6
10,Czech Republic,2.63,7
78,Hong Kong,2.73,8
79,Macao,3.13,9
44,Iceland,3.15,10


****
# Exercise2: 10 cities with the largest population

In [16]:
cities = []         # list to hold city names
populations = []    # list to hold city populations

# Find elements tagged with 'city', iterate over all such elements
for city in root.getiterator(tag='city'):
    
    # Add country name to the list    
    cities.append(city.find('name').text)
    
    # Obtain the latest population information
    years = []    # list to hold years that population data are available
    pops = {}     # dict to hold populations for those years
    for pop in city.findall('population'):
        year = pop.get('year')
        years.append(int(year))
        pops[year] = int(pop.text)
    
    # If data available, get the latest, if not, insert 0
    if years != []:
        populations.append(pops[str(max(years))])
    else:
        populations.append(None)

In [17]:
# Create DataFrame with the above list, list largest top10 cities
d2 = {'City': cities, 'Population': populations}
city_populations = pd.DataFrame(d2).dropna()     # Drop data points with None (NaN)
city_populations = city_populations.sort_values('Population', ascending=False).head(10)
city_populations['Rank'] = [i for i in range(1, 11)]
city_populations

,City,Population,Rank
1341,Shanghai,22315474.0,1
771,Istanbul,13710512.0,2
1527,Mumbai,12442373.0,3
479,Moskva,11979529.0,4
1340,Beijing,11716620.0,5
2810,São Paulo,11152344.0,6
1342,Tianjin,11090314.0,7
1064,Guangzhou,11071424.0,8
1582,Delhi,11034555.0,9
1067,Shenzhen,10358381.0,10


****
# Exercise3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [18]:
data = {}

for country in root.getiterator(tag='country'):
    country_name = country.find('name').text
    years = []
    pops = {}
    for pop in country.findall('population'):
        year = pop.get('year')
        years.append(int(year))
        pops[year] = int(pop.text)
    if years != []:
        country_population = pops[str(max(years))]
    else:
        country_population = 0
    
    egroup = country.findall('ethnicgroup')
    for e in egroup:
        if e.text not in data.keys():
            data[e.text] = 0
        data[e.text] += country_population * float(e.get('percentage')) / 100

In [19]:
ethnic_data = pd.Series(data)
d2 = (ethnic_data/100000).round(1).sort_values(ascending=False).head(10)
d2= d2.to_frame('Population (millions)')
d2['Rank'] = [i for i in range(1, 11)]
d2

,Population (millions),Rank
Han Chinese,12450.6,1
Indo-Aryan,8718.2,2
European,4948.7,3
African,3183.3,4
Dravidian,3027.1,5
Mestizo,1577.3,6
Bengali,1467.8,7
Russian,1318.6,8
Japanese,1265.3,9
Malay,1219.9,10


****
# Exercise4: name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [20]:
# This time, I dealt with try/except to deal with data that are missing/not available
# Besides that, the approach here is the same as above

In [21]:
# a) Longest Rivers
rivers = []
for river in root.getiterator(tag='river'):
    try:
        river_name = river.find('name').text
    except:
        river_name = None
        
    try:
        length = int(river.find('length').text)
    except:
        length = 0
    try:
        country = river.get('country')
    except:
        country = None
    rivers.append({'River name': river_name, 'country': country, 'Length (km)': length})

In [22]:
river_data = pd.DataFrame(rivers)
d3a = river_data.sort_values('Length (km)', ascending=False).head(10)
d3a['Rank'] = [i for i in range(1, 11)]
d3a

,Length (km),River name,country,Rank
174,6448,Amazonas,CO BR PE,1
137,6380,Jangtse,CN,2
136,4845,Hwangho,CN,3
123,4400,Lena,R,4
205,4374,Zaire,RCB ZRE,5
138,4350,Mekong,CN LAO THA K VN,6
115,4248,Irtysch,R KAZ CN,7
186,4184,Niger,RMM RN WAN RG,8
160,4130,Missouri,USA,9
119,4092,Jenissej,R,10


In [23]:
# b) Largest Lakes
lakes = []
for lake in root.getiterator(tag='lake'):
    try:
        lake_name = lake.find('name').text
    except:
        lake_name = None
        
    try:
        area = int(lake.find('area').text)
    except:
        area = 0
    try:
        country = lake.get('country')
    except:
        country = None
    lakes.append({'Lake name': lake_name, 'country': country, 'Area (km^2)': area})

In [24]:
lake_data = pd.DataFrame(lakes)
d3b = lake_data.sort_values('Area (km^2)', ascending=False).head(10)
d3b['Rank'] = [i for i in range(1, 11)]
d3b

,Area (km^2),Lake name,country,Rank
54,386400,Caspian Sea,R AZ KAZ IR TM,1
109,82103,Lake Superior,CDN USA,2
81,68870,Lake Victoria,EAT EAK EAU,3
106,59600,Lake Huron,CDN USA,4
108,57800,Lake Michigan,USA,5
47,41650,Dead Sea,IL JOR WEST,6
83,32893,Lake Tanganjika,ZRE Z BI EAT,7
98,31792,Great Bear Lake,CDN,8
43,31492,Ozero Baikal,R,9
89,29600,Lake Malawi,MW MOC EAT,10


In [25]:
# c) Highest airports
airports = []
for ap in root.getiterator(tag='lake'):
    try:
        ap_name = ap.find('name').text
    except:
        ap_name = None
        
    try:
        elevation = int(ap.find('elevation').text)
    except:
        area = 0
    try:
        country = ap.get('country')
    except:
        country = None
    airports.append({'Airport name': ap_name, 'Country': country, 'Elevation (m)': elevation})

In [26]:
ap_data = pd.DataFrame(airports)
d4b = ap_data.sort_values('Elevation (m)', ascending=False).head(10)
d4b['Rank'] = [i for i in range(1, 11)]
d4b

,Airport name,Country,Elevation (m),Rank
134,Licancabur Crater Lake,RCH BOL,5600,1
64,Nam Co,CN,4718,2
131,Lake Titicaca,BOL PE,3812,3
132,Poopo,BOL,3686,4
133,Salar de Uyuni,BOL,3650,5
58,Koli Sarez,TAD,3250,6
128,Lake Irazu,CR,3200,7
63,Qinghai Lake,CN,3195,8
62,Segara Anak,RI,2008,9
118,Lake Tahoe,USA,1900,10
